In [2]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.simplefilter(action='ignore')

#### **Damped Local Trend (DLT)**

* global and local trend terms

In [3]:
import pandas as pd
import numpy as np
import orbit
from IPython.display import display

In [4]:
display(orbit.__version__)

'1.1.0'

In [5]:
from orbit.utils.dataset import load_iclaims

us_unemp_claims_df = load_iclaims(transform=True).sort_values(by=['week'],ascending=True)
display(us_unemp_claims_df.head())

,week,claims,trend.unemploy,trend.filling,trend.job,sp500,vix
0,2010-01-03,13.386595,0.219882,-0.318452,0.117500,-0.417633,0.122654
1,2010-01-10,13.624218,0.219882,-0.194838,0.168794,-0.425480,0.110445
2,2010-01-17,13.398741,0.236143,-0.292477,0.117500,-0.465229,0.532339
3,2010-01-24,13.137549,0.203353,-0.194838,0.106918,-0.481751,0.428645
4,2010-01-31,13.196760,0.134360,-0.242466,0.074483,-0.488929,0.487404


##### Train Test Split

* Use last year (52 weeks) of data as *Test*, before that as *Train*

In [15]:
import toml, os

parent_dir = os.path.dirname(os.getcwd())
config = toml.load(os.path.join(parent_dir,'config.toml'))
TIME_COL = config['dlt_uber_provided_example']['utils']['TIME_COL']
RESPONSE_COL = config['dlt_uber_provided_example']['utils']['RESPONSE_COL']
test_ind = config['dlt_uber_provided_example']['utils']['test_ind']
train_df, test_df = us_unemp_claims_df[:-test_ind], us_unemp_claims_df[-test_ind:]

#### Three Wrappers for DLT Models

1. ```DLTMAP``` - DLT model for MAP (Maximum a Posteriori)
2. ```DLTFull``` - DLT model for full prediction
    * the prediction occurs as a function of each parameter posterior sample
    * the prediction results are aggregated after prediction
    * prediction will always return the median (aka 50th percentile) along with any additional percentiles that are specified
3. ```DLTAggregated``` - DLT model for aggregated posterior prediction
    * the parameter posterior samples are reduced using aggregate_method ```'mean', 'median'``` before performing a single prediction

In [20]:
from orbit.models import DLT

dlt_map = DLT(
    response_col=RESPONSE_COL,
    date_col=TIME_COL,
    estimator=config['dlt_uber_provided_example']['dlt_map']['estimator'],
    seasonality=config['dlt_uber_provided_example']['dlt_map']['seasonality'],
    seed=config['dlt_uber_provided_example']['utils']['seed']
)